In [1]:
%qtconsole

In [2]:
import pandas as pd
import os
import re
import romkan
import unicodedata
import csv

In [3]:
# get irregular words from ../data/irregular/irregular.csv and create a dict to avoid those words

In [4]:
vocab = pd.read_csv('../data/created/JMdict_e.csv', sep='\t')
kanji = pd.read_csv('../data/created/kanjidic2.csv')

In [5]:
# Run once
del vocab['pos']
del vocab['meaning']
del kanji['Unnamed: 0']
vocab.pronunciation = vocab.pronunciation.map(lambda x: romkan.to_hiragana(romkan.to_roma(x)))
kanji.on = kanji.on.map(lambda x: re.findall("""[\u3040-\u309f]+""", romkan.to_hiragana(romkan.to_roma(x))))
##############################
#kanji.kun = kanji.kun.map(lambda x: list(set(map(lambda y: y.split('.')[0],re.findall("""[\u3040-\u309f.]+""", x))))) # THIS THREW AWAY GOOD INFO
##############################
kanji['kun1'] = kanji.kun.map(lambda x: list(set(map(lambda y: y.split('.')[0],re.findall("""[\u3040-\u309f.]+""", x)))))
kanji['kun2'] = kanji.kun.map(lambda x: list(set(map(lambda y: ''.join(y.split('.')),re.findall("""[\u3040-\u309f.]+""", x)))))
# I should also include the kun without splitting somehow... aka shi.ku = shi and shiku
kanji.nanori = kanji.nanori.map(lambda x: re.findall("""[\u3040-\u309f]+""", x))
kanji.meaning = kanji.meaning.map(lambda x: re.findall("""[a-zA-Z-]+ *[a-zA-Z-]+""", x))
#char_ser = pd.Series(kanji.on.values,index=kanji.character).to_dict()
kanji['onkun'] = kanji.kun1 + kanji.kun2 + kanji.on
#kanji.onkun.map(lambda x: list(set(x))) ########## NOW MAKE SURE ALL ARE SET TO LISTS SO THERE ARE NO DUPLICATES
char_ser = pd.Series(kanji.onkun.values,index=kanji.character).to_dict()
kanji.head()

,character,freq,meaning,on,kun,nanori,kun1,kun2,onkun
0,亜,1509.0,"[Asia, rank next, come after, -ous]",[あ],['つ.ぐ'],"[や, つぎ, つぐ]",[つ],[つぐ],"[つ, つぐ, あ]"
1,唖,NaN,"[mute, dumb]","[あ, あく]",['おし'],[],[おし],[おし],"[おし, おし, あ, あく]"
2,娃,NaN,[beautiful],"[あ, あい, わ]",['うつく.しい'],[い],[うつく],[うつくしい],"[うつく, うつくしい, あ, あい, わ]"
3,阿,1126.0,"[Africa, flatter, fawn upon, corner, nook, rec...","[あ, お]","['おもね.る', 'くま']","[ほとり, あず, あわ, おか, きた, な]","[くま, おもね]","[おもねる, くま]","[くま, おもね, おもねる, くま, あ, お]"
4,哀,1715.0,"[pathetic, grief, sorrow, pathos, pity, sympat...",[あい],"['あわ.れ', 'あわ.れむ', 'かな.しい']",[],"[あわ, かな]","[あわれむ, かなしい, あわれ]","[あわ, かな, あわれむ, かなしい, あわれ, あい]"


In [6]:
#vocab.vocab = vocab.vocab.map(lambda x: romkan.to_hiragana(romkan.to_roma(x)))
#vocab.head()

#char_ser['亜']

#print(jlpt[jlpt.Kanji.str.contains('風')])
#print(char_ser['風'])

In [7]:
irregular = pd.read_csv('../data/irregular/irregular.csv')
jlpt = pd.read_csv('../data/vocab/jlptvocab.csv')
del irregular['URL']

In [8]:
irregular.head()

,Link
0,愛斯蘭
1,銅
2,欠伸
3,飛魚
4,麻布


In [9]:
jlpt = jlpt[jlpt.Kanji.isnull() == 0]
jlpt = jlpt[jlpt.Hiragana.isnull() == 0]
jlpt = jlpt[jlpt.English.isnull() == 0]
jlpt.tail()

,Kanji,Hiragana,English
8334,忘れ物,わすれもの,lost article
8335,笑う,わらう,"to laugh,to smile"
8336,割合,わりあい,"rate,ratio,percentage"
8337,割れる,われる,to break
8353,消しゴム,けしゴム,"eraser, rubber"


In [10]:
def unvoice(char):
    if not ('\u3041' <= char <= '\u3096') or len(char) > 1 or not unicodedata.decomposition(char):
        return char
    return unicodedata.normalize('NFD', char)[0]

In [11]:
### DONT RUN ###
#def check_word(word,cur,pron,pastprons,pastpron):
#    if cur > len(word):
#        return pron
#    try:
#        prons = set(char_ser[word[cur]])
#    except:
#        print(word, ' is not in the dictionary')
#    
#    past_prons.append('')
####

In [12]:
# if prons[-1] == ん: prons = [prons,prons[:-1]]
# if 
def recurs(kanji, word, pron_list = [], done = False):
    if len(pron_list) == len(kanji) or len(word) == 0: # check for 々 and repeat past syllable (unsounded)
        return pron_list, False
    cur = kanji[len(pron_list)]
    if re.match("""[\u3040-\u30ff]""", cur) != None: #should we romkan to hiragana?? ALSO HYPHEN
        if cur != word[0]:
            return pron_list, False
        prons = [cur]
    elif cur == '々':
        if unvoice(pron_list[-1]) == unvoice(word[:len(pron_list[-1])]): # Sou Zou -- didn't catch!!
            pron_list.append(word[:len(pron_list[-1])])
            return pron_list, len(kanji) == len(pron_list)
        else:
            return None, False
    else:
        prons = list(set(char_ser[cur])) # [char_ser[cur]] ###.sort(key=len, reverse=True)
    for pron in prons:        # GET RID OF THIS?
        if pron[-1] == 'ん'and pron != 'ん':
            if isinstance(prons,str):
                prons = [pron, pron[:-1]]
            elif pron[:-1] not in prons:
                prons.append(pron[:-1])
    for pron in prons:
        wordmatch = word[:len(pron)] ###-1
        wordmatch = unvoice(wordmatch[0]) + wordmatch[1:]
        words = [wordmatch, word[:len(pron)]] ### -1
        #print(kanji, word, pron_list, words)
        if pron in words:
            pron_list.append(word[:len(pron)])
            word = word[len(pron):]
            if len(pron_list) == len(kanji) and len(word) == 0:
                return pron_list, True
            new_pron_list, done = recurs(kanji,word,pron_list,done)
            if done:
                return new_pron_list, done
            word = pron_list.pop() + word
    return None, False

# update with unvoice the first syllable of word


# PRONUNCIATION DOES NOT MATCH ON or KUN YOMIKATA... MOSTLY DROPPING LAST N??? -------------

In [13]:
list(set(char_ser['擽'])) #中腹 三日月
#'boy'[1:]
#recurs('相変わらず','あいかわらず',[])
#print(recurs('指輪','ゆびわ',[]))
#char_ser['一']#.sort(key=len, reverse=True)    # EITHER INCLUDE NANORI OR FIX THE ABOVE

['うつ', 'りゃく', 'れき', 'くすぐ', 'う', 'ふき', 'くすぐる', 'らく']

In [8]:
true_cnt = 0
false_cnt = 0
false_dict = {}
true_dict = {}

for _,line in jlpt.iterrows():
    kanji = line.Kanji
    word = line.Hiragana
    #print(kanji,word) ###
    a,b = recurs(kanji,word,[])
    #print(a,b) ###
    if a == None:
        false_cnt += 1
        false_dict[kanji] = word
    else:
        true_cnt += 1
        true_dict[kanji] = a
print(true_cnt, false_cnt) #~8% are irregular...
print(false_dict)

6707 578
{'結核': 'けっかく', '決定': 'けってい', '発想': 'はっそう', '急かす': 'せかす', '学級': 'がっきゅう', '学校': 'がっこう', '他人': 'あだびと', '鈍い': 'のろい', '脱する': 'だっする', '噛る': 'かじる', '咄嗟': 'とっさ', '列島': 'れっとう', '達者': 'たっしゃ', '集る': 'あつまる', '復活': 'ふっかつ', '中学校': 'ちゅうがっこう', '真中': 'まんなか', '凡ゆる': 'あらゆる', '現われ': 'あらわれ', '煙草': 'たばこ', '梅雨': 'つゆ', '迚も': 'とても', '其れから': 'それから', '如何しても': 'どうしても', '芝生': 'しばふ', '知合い': 'しりあい', '平均': 'ならし', '何時も': 'いつも', '一向': 'ひたすら', '素人': 'しろうと', 'お菜': 'おかず', '五月蝿い': 'うるさい', '本当': 'ほんと', '悪戯': 'いたずら', '明い': 'あかるい', '女子': 'おなご', '時々': 'ときどき', '可哀想': 'かわいそう', '含嗽': 'うがい', '物語': 'ものがたり', '末': 'うら', '決行': 'けっこう', '出品': 'しゅっぴん', '可成': 'かなり', '言う': 'ゆう', '延いては': 'ひいては', '殿': 'しんがり', '実際': 'じっさい', '拐う': 'さらう', '血管': 'けっかん', '湿気る': 'しける', '特権': 'とっけん', '浴衣': 'ゆかた', '何方': 'どなた', '様々': 'さまざま', '浮気': 'うわき', '旧事': 'くじ', '田ぼ': 'たんぼ', 'お姉さん': 'おねえさん', '此れ': 'これ', '否': 'いいえ', '発掘': 'はっくつ', '呆気ない': 'あっけない', '不味い': 'まずい', '一括': 'いっかつ', '発見': 'はっけん', '厭やらしい': 'いやらしい', '北極': 'ほっきょく', '草臥れる': 'くたびれる', '心地': 'ここち', '学期':

In [41]:
set(false_dict.keys()) - set(irregular.Link.tolist())

{'お兄さん',
 'お土産',
 'お姉さん',
 'お巡りさん',
 'お早う',
 'お母さん',
 'お洒落',
 'お父さん',
 'お祖母さん',
 'お祖父さん',
 'お腹',
 'お菜',
 'お襁褓',
 'にも拘らず',
 '一体',
 '一切',
 '一向',
 '一変',
 '一定',
 '一家',
 '一寸',
 '一層',
 '一帯',
 '一心',
 '一括',
 '一挙に',
 '一敗',
 '一斉',
 '一方',
 '一日',
 '一旦',
 '一昨年',
 '一昨昨日',
 '一気',
 '一生',
 '一生懸命',
 '一瞬',
 '一種',
 '一筋',
 '一緒',
 '一致',
 '一般',
 '七日',
 '三日月',
 '上る',
 '下す',
 '下る',
 '下手',
 '不味い',
 '不山戯る',
 '中学校',
 '中腹',
 '乗換',
 '乾',
 '予言',
 '二十日',
 '二十歳',
 '二十（歳）',
 '二日',
 '云々',
 '五月蝿い',
 '些とも',
 '今日',
 '今朝',
 '他人',
 '他（外）',
 '仮令',
 '伯父',
 '何々',
 '何の',
 '何れ',
 '何処',
 '何処か',
 '何卒',
 '何故',
 '何故なら',
 '何方',
 '何時か',
 '何時でも',
 '何時の間にか',
 '何時までも',
 '何時も',
 '余所',
 '余所見',
 '余程',
 '作家',
 '作曲',
 '例',
 '俄か',
 '借金',
 '傑作',
 '先々月',
 '八百屋',
 '其の儘',
 '其れ',
 '其れから',
 '其れで',
 '其れでは',
 '其れでも',
 '其れに',
 '其れ共',
 '其れ故',
 '其れ程',
 '其処',
 '其処で',
 '其処ら',
 '其方',
 '凡ゆる',
 '出世',
 '出勤',
 '出品',
 '出席',
 '出張',
 '出版',
 '出生',
 '出産',
 '出発',
 '出社',
 '出血',
 '出費',
 '出身',
 '分る',
 '切手',
 '切符',
 '切開',
 '列島',
 '列車',
 '別荘',
 '前',
 '剥す',
 '副',
 '勉強',
 '北

In [40]:
#true_dict['言う']
#import csv

#my_dict = {"test": 1, "testing": 2}

#with open('falsedict.csv','w') as f:
#    w = csv.writer(f)
#    w.writerows(false_dict.items())

#with open('truedict.csv','w') as f:
#    w = csv.writer(f)
#    w.writerow(['Kanji','Pronunciation'])
#    w.writerows(true_dict.items())

import pickle

#true_dict.to_pickle()
pickle.dump( true_dict, open( "true.p", "wb" ) )

In [19]:
true_dict.keys() & false_dict.keys()

{'一定',
 '一日',
 '七日',
 '三日月',
 '上る',
 '下る',
 '乾',
 '今日',
 '他人',
 '何々',
 '例',
 '前',
 '勉強',
 '十',
 '否',
 '唯',
 '商人',
 '国境',
 '土産',
 '女子',
 '姉妹',
 '少女',
 '平均',
 '年月',
 '弟',
 '悪口',
 '所々',
 '掃除',
 '擦る',
 '散歩',
 '明後日',
 '暖かい',
 '月日',
 '木綿',
 '末',
 '本当',
 '東',
 '梅雨',
 '梯子',
 '洒落',
 '流行',
 '火傷',
 '灰',
 '眼鏡',
 '私',
 '突く',
 '紅葉',
 '練習',
 '背負う',
 '言う',
 '誰',
 '身体',
 '近々',
 '途中',
 '鈍い',
 '館'}

In [ ]:
false_dict['言う']

In [ ]:
#jlpt[jlpt.Kanji.str.contains('亜')]
#jlpt[jlpt.Kanji.isnull()]
print(recurs('十一人','じゅういちにん',[]))
print(recurs('一人','ひとり',[]))
#print(jlpt)

In [ ]:
char_on = pd.Series(kanji.on.values,index=kanji.character).to_dict()
char_kun = pd.Series(kanji.kun.values,index=kanji.character).to_dict()

In [ ]:
#jlpt[jlpt.Kanji.str.contains('亜')]
jlpt.head()

In [ ]:
### THIS WAS A PREVIOUS TEST
word = '漢数字'
pron = 'かんすうじ'
for ji in word:
    pron_list = re.findall("""[\u3040-\u309f]+""", char_ser[ji])
    pron_list.sort(key=len, reverse=True)
    print(pron_list)
    for pr in pron_list:
        if pron[0:len(pr)] == pr: # re match beginning of string
            print(ji, pr)
            pron = pron[len(pr):]
    #re.match()
    
#next match up each kanji to on or kun for exploratory analysis

#Maybe if word is only kanji - first check on list, if word is mixture - first check kun?

# Also put in checks for hirgana matching ()